In [1]:
import pathlib
from Utils.wetland_selection    import get_top_scenes
from Utils.treatment_regression import logistic_process
from Utils.outcome_syndata_generator import generate_synthetic_outcome
from Utils.treatment_syndata_generator import generate_synthetic_treatment
from Utils.outcome_regression import regression_process
from Utils.convolution import make_exp_kernel
from Utils.outcome_post import generate_post
import numpy as np

In [3]:
# top scenes by wetland development
top_df = get_top_scenes(num_scenes=3566)
scene_ids = top_df['scene'].astype(str).tolist()

# define folder paths for rasters
DATA_DIR = pathlib.Path('/Users/sayedmorteza/Library/CloudStorage/Box-Box/Hetwet_Data')         

# folders
folders = {
    'wet': DATA_DIR / 'WETLAND_DEV_1996_2016',
    'dem': DATA_DIR / 'DEM',
    'cap': DATA_DIR / 'CAPITAL_1996',
    'claims_96': DATA_DIR / 'LOG_CLAIMS_1996',
    'claims_16': DATA_DIR / 'LOG_CLAIMS_2016',

}

In [4]:
# estimate treatment logistic regression parameters
theta = 0.6  # classification threshold
logit_res = logistic_process(
    scene_ids=scene_ids,
    folders=folders,
    threshold=theta,
    regularization_C=1.0,
    noise_type='gaussian',
    noise_sd=0.01
)

INFO: Processing scene 3043
INFO: Processing scene 2985
INFO: Processing scene 3095
INFO: Processing scene 2779
INFO: Processing scene 2922
INFO: Processing scene 3120
INFO: Processing scene 3044
INFO: Processing scene 3138
INFO: Processing scene 3088
INFO: Processing scene 2953
INFO: Processing scene 3186
INFO: Processing scene 2672
INFO: Processing scene 1934
INFO: Processing scene 2137
INFO: Processing scene 2954
INFO: Processing scene 2115
INFO: Processing scene 2530
INFO: Processing scene 3163
INFO: Processing scene 1962
INFO: Processing scene 2921
INFO: Processing scene 628
INFO: Processing scene 3113
INFO: Processing scene 2715
INFO: Processing scene 2126
INFO: Processing scene 2483
INFO: Processing scene 2032
INFO: Processing scene 1964
INFO: Processing scene 3069
INFO: Processing scene 3087
INFO: Processing scene 1993
INFO: Processing scene 1935
INFO: Processing scene 2171
INFO: Processing scene 2086
INFO: Processing scene 2716
INFO: Processing scene 2023
INFO: Processing scen

In [5]:
# generate synthetic treatment
generate_synthetic_treatment(
    scene_ids=scene_ids,
    folders=folders,
    logit_pipe=logit_res,
    threshold=theta,
    noise_sd=0.01,
    noise_type='gaussian',
)


INFO: [gaussian] Saved synthetic for 3043
INFO: [gaussian] Saved synthetic for 2985
INFO: [gaussian] Saved synthetic for 3095
INFO: [gaussian] Saved synthetic for 2779
INFO: [gaussian] Saved synthetic for 2922
INFO: [gaussian] Saved synthetic for 3120
INFO: [gaussian] Saved synthetic for 3044
INFO: [gaussian] Saved synthetic for 3138
INFO: [gaussian] Saved synthetic for 3088
INFO: [gaussian] Saved synthetic for 2953
INFO: [gaussian] Saved synthetic for 3186
INFO: [gaussian] Saved synthetic for 2672
INFO: [gaussian] Saved synthetic for 1934
INFO: [gaussian] Saved synthetic for 2137
INFO: [gaussian] Saved synthetic for 2954
INFO: [gaussian] Saved synthetic for 2115
INFO: [gaussian] Saved synthetic for 2530
INFO: [gaussian] Saved synthetic for 3163
INFO: [gaussian] Saved synthetic for 1962
INFO: [gaussian] Saved synthetic for 2921
INFO: [gaussian] Saved synthetic for 628
INFO: [gaussian] Saved synthetic for 3113
INFO: [gaussian] Saved synthetic for 2715
INFO: [gaussian] Saved synthetic fo

In [6]:
# estimate outcome linear regression parameters
outcome_reg = regression_process(
    scene_ids=scene_ids,
    folders=folders,
    noise_sd=0.1,
    noise_type='gaussian'
)

INFO: Loading scene 3043
INFO: Loading scene 2985
INFO: Loading scene 3095
INFO: Loading scene 2779
INFO: Loading scene 2922
INFO: Loading scene 3120
INFO: Loading scene 3044
INFO: Loading scene 3138
INFO: Loading scene 3088
INFO: Loading scene 2953
INFO: Loading scene 3186
INFO: Loading scene 2672
INFO: Loading scene 1934
INFO: Loading scene 2137
INFO: Loading scene 2954
INFO: Loading scene 2115
INFO: Loading scene 2530
INFO: Loading scene 3163
INFO: Loading scene 1962
INFO: Loading scene 2921
INFO: Loading scene 628
INFO: Loading scene 3113
INFO: Loading scene 2715
INFO: Loading scene 2126
INFO: Loading scene 2483
INFO: Loading scene 2032
INFO: Loading scene 1964
INFO: Loading scene 3069
INFO: Loading scene 3087
INFO: Loading scene 1993
INFO: Loading scene 1935
INFO: Loading scene 2171
INFO: Loading scene 2086
INFO: Loading scene 2716
INFO: Loading scene 2023
INFO: Loading scene 2891
INFO: Loading scene 2277
INFO: Loading scene 2813
INFO: Loading scene 1992
INFO: Loading scene 1775
I

In [7]:
# generate synthetic outcomes:
generate_synthetic_outcome(
    scene_ids = scene_ids,
    folders = folders,
    reg_pipe = outcome_reg,
    target_shape = (8, 8),
    noise_type = "gaussian",
    noise_sd = 0.1,
    results_dir = None
)

INFO: [3043] Saved synthetic → scene_3043_synthetic_gaussian.tiff
INFO: [2985] Saved synthetic → scene_2985_synthetic_gaussian.tiff
INFO: [3095] Saved synthetic → scene_3095_synthetic_gaussian.tiff
INFO: [2779] Saved synthetic → scene_2779_synthetic_gaussian.tiff
INFO: [2922] Saved synthetic → scene_2922_synthetic_gaussian.tiff
INFO: [3120] Saved synthetic → scene_3120_synthetic_gaussian.tiff
INFO: [3044] Saved synthetic → scene_3044_synthetic_gaussian.tiff
INFO: [3138] Saved synthetic → scene_3138_synthetic_gaussian.tiff
INFO: [3088] Saved synthetic → scene_3088_synthetic_gaussian.tiff
INFO: [2953] Saved synthetic → scene_2953_synthetic_gaussian.tiff
INFO: [3186] Saved synthetic → scene_3186_synthetic_gaussian.tiff
INFO: [2672] Saved synthetic → scene_2672_synthetic_gaussian.tiff
INFO: [1934] Saved synthetic → scene_1934_synthetic_gaussian.tiff
INFO: [2137] Saved synthetic → scene_2137_synthetic_gaussian.tiff
INFO: [2954] Saved synthetic → scene_2954_synthetic_gaussian.tiff
INFO: [211

In [8]:
# build your spillover kernel
CELL_SIZE = 960 # cell size in meters
LAMBDA    = 5000.0
KERNEL    = make_exp_kernel(lam_m=LAMBDA, cell_size_m=CELL_SIZE, truncate=4)

In [9]:
generate_post(
    scene_ids=scene_ids,
    folders=folders,
    KERNEL=KERNEL,
    Bbase = 0.02,
    Beta1 = 0.1,
    Beta2 = 1
)

/opt/miniconda3/envs/imageTRT/lib/python3.10/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
INFO: [3043] Saved post outcome coarse → scene_3043_post_gaussian.tiff
INFO: [3043] Saved ITE coarse → scene_3043_ITE.tiff
INFO: [2985] Saved post outcome coarse → scene_2985_post_gaussian.tiff
INFO: [2985] Saved ITE coarse → scene_2985_ITE.tiff
INFO: [3095] Saved post outcome coarse → scene_3095_post_gaussian.tiff
INFO: [3095] Saved ITE coarse → scene_3095_ITE.tiff
INFO: [2779] Saved post outcome coarse → scene_2779_post_gaussian.tiff
INFO: [2779] Saved ITE coarse → scene_2779_ITE.tiff
INFO: [2922] Saved post outcome coarse → scene_2922_post_gaussian.tiff
INFO: [2922] Saved ITE coarse → scene_2922_ITE.tiff
INFO: [3120] Saved post outcome coarse → scene_3120_post_gaussian.tiff
INFO: [3120] Saved ITE coarse → scene_3120_ITE.t